In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, LeakyReLU, ReLU
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy.stats import pearsonr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data preparation

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/IMD-TP Load Forecast/Updated Yearly Data/allyears.csv')

In [ ]:
data.shape

(195553, 4)

In [ ]:
data

,DateTime,RH,Temp,Load
0,01-01-2018 00:00,80.423,18.160,174.542804
1,01-01-2018 00:15,81.471,18.064,175.981604
2,01-01-2018 00:30,82.675,17.927,178.636804
3,01-01-2018 00:45,84.552,17.745,180.045604
4,01-01-2018 01:00,86.663,17.751,182.254000
...,...,...,...,...
195548,30-07-2023 23:00,91.780,27.740,298.124610
195549,30-07-2023 23:15,92.100,27.660,295.327468
195550,30-07-2023 23:30,92.420,27.620,291.813111
195551,30-07-2023 23:45,92.320,27.640,288.324763


In [ ]:
# data = np.delete(data, 0, axis=1)
# # data = np.delete(data, 1, axis=1)
# # data = np.delete(data, 3, axis=1)

In [ ]:
# Specify the columns you want to delete
columns_to_delete = ['DateTime','Load']  # Replace with the actual column names you want to delete

# Drop the specified columns
data.drop(columns=columns_to_delete, inplace=True)

In [ ]:
data.head()

,RH,Temp
0,80.423,18.160
1,81.471,18.064
2,82.675,17.927
3,84.552,17.745
4,86.663,17.751


In [ ]:
data.to_csv("/content/drive/MyDrive/IMD-TP Load Forecast/Updated Yearly Data/allyearsRHTemp.csv",index=False)

# Preprocessing

In [ ]:
RHTemp = pd.read_csv("/content/drive/MyDrive/IMD-TP Load Forecast/Updated Yearly Data/allyearsRHTemp.csv")

In [ ]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(RHTemp)

In [ ]:
scaled_data

array([[0.54983716, 0.19454254],
       [0.55858873, 0.19146067],
       [0.56864301, 0.1870626 ],
       ...,
       [0.65002088, 0.49823435],
       [0.6491858 , 0.4988764 ],
       [0.64734864, 0.49951846]])

In [ ]:
scaled_data.shape

(195553, 2)

In [ ]:
# Preprocess the data
n_steps_in = 96 * 7  # Number of steps to take as input
n_steps_out = 192  # Number of steps to predict


# Reshape the data into input and target sequences
X, y = [], []
for i in range(n_steps_in, len(scaled_data) - 192):
    X.append(scaled_data[i - n_steps_in:i])
    y.append(scaled_data[i:i + 192,1])  # Extract the first column of the target data

X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

(194689, 672, 2)

In [ ]:
y.shape

(194689, 192)

In [ ]:
split_index = int(0.96 * len(X))  # 96% for training, 4% for testing
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
len(scaled_data)

195553

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(186901, 672, 2)
(7788, 672, 2)
(186901, 192)
(7788, 192)


# Denormalizing

In [ ]:
# Denormalising or Rescaling
tempy = []
for i in range(672 ,RHTemp.shape[0] - 192):
  tempy.append(RHTemp[i:i+192])
tempy = np.array(tempy)
y_train_og, y_test_og = train_test_split(tempy,test_size=0.04,shuffle=False)

In [ ]:
y_train_og.shape

(186901, 192, 2)

In [ ]:
y_test_og.shape

(7788, 192, 2)

In [ ]:
tempy.shape

(194689, 192, 2)

In [ ]:
y_test_og = y_test_og.reshape(y_test_og.shape[0],y_test_og.shape[1],y_test_og.shape[2])
y_test_og.shape

(7788, 192, 2)

In [ ]:
tempy=tempy.reshape(tempy.shape[0],tempy.shape[1]*tempy.shape[2])
tempy.shape

(194689, 384)

In [ ]:
# y_test_og = y_test_og.reshape(y_test_og.shape[0],y_test_og.shape[1],y_test_og.shape[2])
# tempy=tempy.reshape(tempy.shape[0],tempy.shape[1]*tempy.shape[2])
scalery = MinMaxScaler()
models=scalery.fit(tempy)
sc_data=models.transform(tempy)

# Model building

In [ ]:
# Configure k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=False)

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(100, activation='tanh',return_sequences=True, input_shape=(n_steps_in, 2)))  # Assuming 2 features
model.add(LSTM(128, activation='tanh', return_sequences=True))
model.add(LSTM(64, activation='tanh'))
model.add(Dense(n_steps_out))  # Output layer with 96 units for load prediction
model.compile(optimizer='adam', loss='mse')  # Use Mean Squared Error as the loss function

# Perform k-fold cross-validation
for train_idx, val_idx in kfold.split(X_train):
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Train the model on the current fold
    history = model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, validation_data=(X_val_fold, y_val_fold))



Epoch 1/10
4673/4673 [==============================] - 342s 71ms/step - loss: 0.0080 - val_loss: 0.0024
Epoch 2/10
4673/4673 [==============================] - 331s 71ms/step - loss: 0.0061 - val_loss: 0.0025
Epoch 3/10
4673/4673 [==============================] - 325s 70ms/step - loss: 0.0058 - val_loss: 0.0018
Epoch 4/10
4673/4673 [==============================] - 324s 69ms/step - loss: 0.0061 - val_loss: 0.0019
Epoch 5/10
4673/4673 [==============================] - 336s 72ms/step - loss: 0.0054 - val_loss: 0.0018
Epoch 6/10
4673/4673 [==============================] - 336s 72ms/step - loss: 0.0055 - val_loss: 0.0017
Epoch 7/10
4673/4673 [==============================] - 322s 69ms/step - loss: 0.0049 - val_loss: 0.0025
Epoch 8/10
4673/4673 [==============================] - 324s 69ms/step - loss: 0.0043 - val_loss: 0.0019
Epoch 9/10
4673/4673 [==============================] - 323s 69ms/step - loss: 0.0039 - val_loss: 0.0018
Epoch 10/10
4673/4673 [==============================] 

# Evaluations

In [ ]:
# Evaluate the model on the test set
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error on Test Set:", mse)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)

In [ ]:
kpcc = pearsonr(y_pred.flatten(), y_test.flatten())
print("Pearson Correlation Coefficient (KPCC):", kpcc)

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

# Model saving

In [ ]:
model.save('/content/drive/MyDrive/IMD-TP Load Forecast/Models/_RH.h5')